In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import requests
import re
import os
import numpy as np
import pandas as pd
chromedriver = "/usr/local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver) #opens a new chrome instance run by selenium
driver.get("https://irma.nps.gov/Stats/MvcReportViewer.aspx?_id=361d40ec-99ad-42ad-b486-97534b990064&_m=Remote&_r=%2fNPS.Stats.Reports%2fPark+Specific+Reports%2fRecreation+Visitors+By+Month+(1979+-+Last+Calendar+Year)&_39=880px&Park=MORA")
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [62]:
park_data = [['Year', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'TOTAL', 'PARKNAME']]

In [63]:
# We want to look at all national parks
np_elements = driver.find_elements_by_xpath("//option[contains(text(),'NP')]")
all_np = []

for i in np_elements:
    all_np.append(i.text)
    
#print(all_np) - this contains all of the places for which the NPS has kept data (including National Parks, National Historic Sites, etc.) 

only_np = []
for u in all_np:
    if u.endswith('NP'):
        only_np.append(u)
# Now we have a list of all of the National Parks by their name!
print(only_np)

['Acadia NP', 'Arches NP', 'Badlands NP', 'Big Bend NP', 'Biscayne NP', 'Black Canyon of the Gunnison NP', 'Bryce Canyon NP', 'Canyonlands NP', 'Capitol Reef NP', 'Carlsbad Caverns NP', 'Channel Islands NP', 'Congaree NP', 'Crater Lake NP', 'Cuyahoga Valley NP', 'Death Valley NP', 'Dry Tortugas NP', 'Everglades NP', 'Glacier NP', 'Grand Canyon NP', 'Grand Teton NP', 'Great Basin NP', 'Great Smoky Mountains NP', 'Guadalupe Mountains NP', 'Haleakala NP', 'Hawaii Volcanoes NP', 'Hot Springs NP', 'Isle Royale NP', 'Joshua Tree NP', 'Kenai Fjords NP', 'Kings Canyon NP', 'Kobuk Valley NP', 'Lassen Volcanic NP', 'Mammoth Cave NP', 'Mesa Verde NP', 'Mount Rainier NP', 'North Cascades NP', 'Olympic NP', 'Petrified Forest NP', 'Pinnacles NP', 'Redwood NP', 'Rocky Mountain NP', 'Saguaro NP', 'Sequoia NP', 'Shenandoah NP', 'Theodore Roosevelt NP', 'Virgin Islands NP', 'Voyageurs NP', 'Wind Cave NP', 'Yellowstone NP', 'Yosemite NP', 'Zion NP']


In [67]:
for p in only_np:
    select = Select(driver.find_element_by_id('ReportViewer_ctl04_ctl03_ddValue'))
    select.select_by_visible_text(p)
    driver.find_element_by_xpath('//input[@type="submit"]').click()
    time.sleep(10)
    # We're going to start by finding the park name, because scraping it for each row doesn't make sense and makes it run way slower:
    select = Select(driver.find_element_by_id("ReportViewer_ctl04_ctl03_ddValue"))
    selected_park = select.first_selected_option.text
    # Now to scrape the data
    for i in np.arange(2018,1978,-1):
        search_for_year = soup.body.find_all(text=i)
        # This finds us the cell with each year, which is the first cell in each row.
        div_of_year = [date.parent for date in search_for_year]
        # This finds us the parent div for that cell
        td_of_year = [div.parent for div in div_of_year]
        # This finds us the parent td for that div
        tr_of_year = [td.parent for td in td_of_year]
        # This finds us the parent tr for that td.  Now we have the whole row's html, so we want to search for the actual text that we need, i.e. the values for the cells.
        # All of the values have the same text before and after it, so we can just search for the string in between these two:
        start = 'style="word-wrap:break-word;white-space:pre-wrap;">'
        end = '</div>'
        def find_values(tr):
            result = re.findall('%s(.*?)%s' % (start, end), tr)
            return(result)
        tr_of_year_s = str(tr_of_year)
        # We need it in string format for findall to work
        values_of_year = find_values(tr_of_year_s)
        # This should give us 14 values for each row. The first column is the year, then the next 12 correspond to each month, and the last is a total
        # For the current year, we don't have 14 values. We have whatever months have transpired and the total, so we'll fill in the other columns with NaN
        while len(values_of_year) != 14:
            values_of_year.insert(-1, 'NaN')
        # Now we should have a row with 14 values, no matter how many were originally filled. Now let's add in the park name
        values_of_year.append(selected_park)
        # Now we just append the whole row!
        park_data.append(values_of_year)
    

In [1]:
#print(park_data)

In [ ]:
np_df = pd.DataFrame(park_data)

In [72]:
np_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Year,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,TOTAL,PARKNAME
1,2018,"23,928","18,782",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"42,710",Acadia NP
2,2017,"27,204","21,557","22,768","34,679","97,809","189,348","329,618","323,130","195,641","89,520","58,164","26,429","1,415,867",Acadia NP
3,2016,"34,786","26,655","28,799","50,395","93,828","187,037","296,922","313,699","195,579","76,232","27,114","25,867","1,356,913",Acadia NP
4,2015,"28,663","18,527","30,565","49,589","69,225","160,878","271,118","259,670","179,756","75,827","71,663","21,750","1,237,231",Acadia NP


In [75]:
np_df.to_csv('NP_DATA.csv', index=True, header=False)

In [7]:
np_df = pd.read_csv('NP_DATA.csv')

In [9]:
np_df.head()

,Year,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,TOTAL,PARKNAME
0,2018,"23,928","18,782",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"42,710",Acadia NP
1,2017,"27,204","21,557","22,768","34,679","97,809","189,348","329,618","323,130","195,641","89,520","58,164","26,429","1,415,867",Acadia NP
2,2016,"34,786","26,655","28,799","50,395","93,828","187,037","296,922","313,699","195,579","76,232","27,114","25,867","1,356,913",Acadia NP
3,2015,"28,663","18,527","30,565","49,589","69,225","160,878","271,118","259,670","179,756","75,827","71,663","21,750","1,237,231",Acadia NP
4,2014,"23,824","10,641","19,935","31,295","55,951","161,513","263,948","320,378","173,827","76,384","111,446","15,117","1,264,259",Acadia NP


In [10]:
np_df.columns

Index(['Year', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'TOTAL', 'PARKNAME'],
      dtype='object')